# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [2]:
import pandas as pd

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [3]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

#### Usamos el metodo **head** para ver como estan estructurados los elementos del JSON

In [4]:
datos.head(10)

,actor,result,verb,_id,object,timestamp
0,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.361000+00:00
1,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'https://w3id.org/xapi/serious...,{'id': 'http://adlnet.gov/expapi/verbs/progres...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.317000+00:00
2,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'extensions': {'fullscreen': True, 'language'...",{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.316000+00:00
3,"{'name': 'gqoj', 'account': {'homePage': 'http...",NaN,{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.297000+00:00
4,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'scene': 'menu'}},{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.439000+00:00
5,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Estrellas por cate...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.435000+00:00
6,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.434000+00:00
7,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Menús: Puedes eleg...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.688000+00:00
8,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.685000+00:00
9,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:46.017000+00:00


#### Para empezar de forma mas sencilla analizamos el tiempo de juego de un jugador concreto, por ejemplo, de *gqoj*

* Lo primero sería extraer su primer *timestamp*  

In [12]:
inicio = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        inicio = i[5]
        break

In [6]:
inicio

Timestamp('2022-05-16 10:36:35.361000+0000', tz='UTC')

* Ahora que ya tenemos el inicio de la partida vamos a buscar el fin de la partida, para ello nos recorremos todo el JSON y nos quedamos con el último *timestamp* del jugador

In [7]:
fin = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        fin = i[5]

In [8]:
fin

Timestamp('2022-05-23 16:01:55.895000+0000', tz='UTC')

Parece que las trazas no son todas del mismo día, hay que buscar la forma de **filtrar por días**.

#### Ahora que ya sabemos extraer el *timestamp* de un jugador en concreto vamos a intentar recoger los de todos los jugadores

* Guardaremos los datos en un diccionario cuya clave será el **id** del jugador y su valor una pareja con el *timestamp* de inicio y de fin

In [13]:
tiempos = dict()
for i in datos.to_numpy():
    idUser = i[0]["name"]
    timeStamp = i[5]
    
    if(idUser in tiempos):
        timeStampIni = tiempos[idUser][0]
        tiempos[idUser] = (timeStampIni, timeStamp)
    else:
        tiempos[idUser] = (timeStamp, "")

In [15]:
pd.DataFrame(tiempos)

,gqoj,hgfx,gton,uala,qeqyj,fkmtp,hjpbw,bdwyj,eprdu,ftpnb,...,bajbm,eayrh,orjyn,prpej,fmupf,vkfnt,xbmmw,pleui,olugf,epjcx
0,2022-05-16 10:36:35.361000+00:00,2022-05-16 10:48:06.634000+00:00,2022-05-16 22:12:06.760000+00:00,2022-05-17 07:39:39.267000+00:00,2022-05-18 07:10:24.601000+00:00,2022-05-18 07:10:44.894000+00:00,2022-05-18 07:10:59.919000+00:00,2022-05-18 07:11:02.294000+00:00,2022-05-18 07:11:01.092000+00:00,2022-05-18 07:11:18.662000+00:00,...,2022-05-24 09:33:01.510000+00:00,2022-05-24 09:33:02.422000+00:00,2022-05-24 09:33:07.855000+00:00,2022-05-24 09:33:41.060000+00:00,2022-05-24 09:34:20.130000+00:00,2022-05-24 09:34:23.401000+00:00,2022-05-24 09:34:22.295000+00:00,2022-05-24 09:34:32.727000+00:00,2022-05-24 09:36:36.250000+00:00,2022-05-24 12:20:00.205000+00:00
1,2022-05-23 16:01:55.895000+00:00,2022-05-16 10:54:01.016000+00:00,2022-05-16 22:12:21.778000+00:00,2022-05-17 07:42:45.832000+00:00,2022-05-18 10:10:50.954000+00:00,2022-05-18 10:10:45.117000+00:00,2022-05-18 10:10:58.339000+00:00,2022-05-18 10:13:10.064000+00:00,2022-05-18 10:10:53.289000+00:00,2022-05-18 10:10:55.723000+00:00,...,2022-05-24 12:13:54.834000+00:00,2022-05-24 12:12:45.295000+00:00,2022-05-24 12:14:02.923000+00:00,2022-05-24 12:09:03.889000+00:00,2022-05-24 12:11:58.891000+00:00,2022-05-24 12:13:18.012000+00:00,2022-05-24 12:12:04.445000+00:00,2022-05-24 12:12:17.773000+00:00,2022-05-24 12:09:25.385000+00:00,2022-05-24 12:58:14.766000+00:00


* Parece que ya tenemos los datos que queríamos de cada jugador, asumimos que son los correctos porque los tiempos del jugador *gqoj* coinciden con los que hemos sacado antes

* Ahora toca hacer la resta para ver el tiempo de juego de los jugadores 